# Search Images by other Images

In [ ]:
import json

from os import makedirs, path
from PIL import Image as PImage

## Calculate Crop Embeddings

In [ ]:
import json

from os import listdir, makedirs, path
from PIL import Image as PImage

from models.CLIP import Clip
from models.Owlv2 import Owlv2Embedding as Owlv2
from models.SigLip2 import SigLip2

In [ ]:
CROP_IMGS_PATH = "../../imgs/arts/crops"
CROP_EMBED_PATH = "./metadata/json/art-crops/embeddings"

# CROP_IMGS_PATH = "../../imgs/palms/crops"
# CROP_EMBED_PATH = "./metadata/json/palm-crops/embeddings"

makedirs(CROP_EMBED_PATH, exist_ok=True)

In [ ]:
model = Clip()
model_name = type(model).__name__.lower().replace("embedding", "")
model_name

In [ ]:
crop_fnames = sorted([fn for fn in listdir(CROP_IMGS_PATH) if fn.endswith(".jpg")])
len(crop_fnames)

In [ ]:
for idx,fname in enumerate(crop_fnames):
  if idx % 100 == 0:
    print(f"{idx} / {len(crop_fnames)}")

  qid = fname.replace(".jpg", "")
  image_path = path.join(CROP_IMGS_PATH, fname)
  embedding_path = path.join(CROP_EMBED_PATH, f"{qid}.json")

  embeds = {}
  if path.isfile(embedding_path):
    with open(embedding_path, "r") as ifp:
      embeds = json.load(ifp)[qid]

  if model_name in embeds:
    continue

  img = PImage.open(image_path)
  embeds[model_name] = [round(v, 8) for v in model.get_embedding(img).tolist()]
  embedding_data = { qid: embeds }

  with open(embedding_path, "w") as ofp:
    json.dump(embedding_data, ofp, separators=(",",":"), sort_keys=True, ensure_ascii=False)

### Test Embeddings

In [ ]:
import json
import numpy as np

from os import listdir, makedirs, path
from PIL import Image as PImage

from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

In [ ]:
PALM_IMGS_PATH = "../../imgs/palms/crops"
PALM_EMBED_PATH = "./metadata/json/palm-crops/embeddings"

model = "siglip2"

target = "caryota-rumphiana_001"

target_embedding_path = path.join(PALM_EMBED_PATH, f"{target}.json")
with open(target_embedding_path, "r") as ifp:
  target_embedding = np.array([json.load(ifp)[target][model]])

In [ ]:
embed_fnames = sorted([fn for fn in listdir(PALM_EMBED_PATH) if fn.endswith(".json")])

In [ ]:
all_embeds = []
all_ids = []

for fname in embed_fnames:
  qid = fname.replace(".json", "")
  embedding_path = path.join(PALM_EMBED_PATH, fname)
  img_path = path.join(PALM_IMGS_PATH, f"{qid}.jpg")

  with open(embedding_path, "r") as ifp:
    all_embeds.append(json.load(ifp)[qid][model])
    all_ids.append(qid)

all_ids = np.array(all_ids)
all_embeds = np.array(all_embeds)

In [ ]:
cos_dists = cosine_distances(all_embeds, target_embedding).reshape(-1)
euc_dists = euclidean_distances(all_embeds, target_embedding).reshape(-1)

cos_dist_idx = np.argsort(cos_dists)
euc_dist_idx = np.argsort(euc_dists)

In [ ]:
all_ids[cos_dist_idx[:3]], all_ids[euc_dist_idx[:3]]

## Search by Image

Note:
- Owlv2 doesn't really work
- Clip is ok
- SigLip2 is best

In [ ]:
import json
import numpy as np

from os import listdir, makedirs, path
from PIL import Image as PImage, ImageOps as PImageOps, ImageDraw as PImageDraw

from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

In [ ]:
PALM_IMGS_PATH = "../../imgs/palms/crops"
PALM_EMBED_PATH = "./metadata/json/palm-crops/embeddings"

model = "siglip2"

target = "ceroxylon_000"

target_embedding_path = path.join(PALM_EMBED_PATH, f"{target}.json")
with open(target_embedding_path, "r") as ifp:
  target_embedding = np.array([json.load(ifp)[target][model]])

tgt_img = PImage.open(path.join(PALM_IMGS_PATH, f"{target}.jpg"))
display(tgt_img)

In [ ]:
CROP_IMGS_PATH = "../../imgs/arts/crops"
CROP_EMBED_PATH = "./metadata/json/art-crops/embeddings"

In [ ]:
embed_fnames = sorted([fn for fn in listdir(CROP_EMBED_PATH) if fn.endswith(".json")])

In [ ]:
all_embeds = []
all_ids = []

for fname in embed_fnames:
  qid = fname.replace(".json", "")
  embedding_path = path.join(CROP_EMBED_PATH, fname)
  img_path = path.join(CROP_IMGS_PATH, f"{qid}.jpg")

  with open(embedding_path, "r") as ifp:
    all_embeds.append(json.load(ifp)[qid][model])
    all_ids.append(qid)

all_ids = np.array(all_ids)
all_embeds = np.array(all_embeds)

In [ ]:
cos_dists = cosine_distances(all_embeds, target_embedding).reshape(-1)
euc_dists = euclidean_distances(all_embeds, target_embedding).reshape(-1)

cos_dist_idx = np.argsort(cos_dists)
euc_dist_idx = np.argsort(euc_dists)

all_ids[cos_dist_idx[:3]], all_ids[euc_dist_idx[:3]]

In [ ]:
for idx in euc_dist_idx[:8]:
  id = all_ids[idx]
  pimg = PImage.open(path.join(CROP_IMGS_PATH, f"{id}.jpg"))
  pimg.thumbnail((200,200))
  print(id)
  display(pimg)

In [ ]:
for idx in cos_dist_idx[:8]:
  id = all_ids[idx]
  pimg = PImage.open(path.join(CROP_IMGS_PATH, f"{id}.jpg"))
  pimg.thumbnail((200,200))
  print(id)
  display(pimg)

## Get Image Crop by Objectness

In [ ]:
from PIL import Image as PImage, ImageDraw as PImageDraw

from models.Owlv2 import Owlv2

In [ ]:
model = Owlv2()

In [ ]:
fname = "borassus_000"
img = PImage.open(f"../../imgs/palms/00/{fname}.jpg")
img.thumbnail((900,900))

In [ ]:
crop_boxes = model.get_objectness_boxes(img, 8)

In [ ]:
dimg = img.copy()
draw = PImageDraw.Draw(dimg)

for x1, y1, x2, y2 in crop_boxes:
  draw.rectangle(xy=((x1, y1), (x2, y2)), outline="red")

display(dimg)

## SigLip2 (embedding)

Large patch 16x16:
- https://huggingface.co/google/siglip2-large-patch16-384
- https://huggingface.co/google/siglip2-large-patch16-512

Giant 16x16:
- https://huggingface.co/google/siglip2-giant-opt-patch16-256
- https://huggingface.co/google/siglip2-giant-opt-patch16-384


## Owl2 (zero-shot detection)
- https://huggingface.co/google/owlv2-base-patch16
- https://huggingface.co/google/owlv2-large-patch14
- https://huggingface.co/google/owlv2-large-patch14-ensemble

- [Niels' Tutorial](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/OWLv2/Zero_and_one_shot_object_detection_with_OWLv2.ipynb)

#### Results from experiments
- Use larger images and smaller models